# A simple feed forward model

```




```
- [Read data](#Read-data)
- [Prepare data](#Prepare-data)
- [Create and train model](#Create-and-train-model)
- [Test on unseen data](#Test-on-unseen-data)

In [1]:
import os
import sys
import json
import pickle

import numpy as np
import pandas as pd

from sklearn.model_selection import GridSearchCV, train_test_split
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [2]:
MODULES_PATH = '../modules'
MODELS_PATH = '../models'
DATA_PATH = '../data'

sys.path.append(MODULES_PATH)
from data import flatten_data, prepare_training_data, prepare_test_data, \
                    raise_one_level
from models import parameter_ffn_seq

## Read data

In [3]:
with open(os.path.join(DATA_PATH,'sentences.json'),'r') as datafile:
    sentences = json.load(datafile)

In [4]:
documents = pd.read_csv('../data/training_data.csv')

## Prepare data

In [5]:
sentences_flat = raise_one_level(sentences)
sentences_df = pd.DataFrame(sentences_flat)

In [6]:
corpora_train, corpora_test, labels_train, labels_test = train_test_split(
                                                                        sentences_df['body'],
                                                                        sentences_df['class'],
                                                                        test_size=0.25,
                                                                        random_state=123)

In [7]:
training_data = []
for i in range(1,2):
    print(i+1)

    document_matrix, labels, pipeline_instance = prepare_training_data(corpora_train, labels_train, (i,i))
    training_data.append({'document_matrix': document_matrix, 'labels': labels, 'pipeline_instance': pipeline_instance})

2


In [30]:
training_data[0]['document_matrix'].shape, training_data[0]['labels'].shape, training_data[0]['pipeline_instance']

((10139, 68), (10139, 11), Pipeline(memory=None,
      steps=[('vect', CountVectorizer(analyzer='char', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 1), preprocessor=None, stop_words=None,
         strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
         tokenizer=None, vocabulary=None)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True))]))

In [29]:
for i in training_data:
    print(i['document_matrix'].shape)

(10139, 68)
(10139, 2161)
(10139, 21082)


In [14]:
with open(os.path.join(DATA_PATH, 'pipeline_instance.pickle'),'wb') as datafile:
        pickle.dump(pipeline_instance, datafile)

## Create and train model

In [8]:
ffn = KerasClassifier(build_fn=parameter_ffn_seq, verbose=1)

In [9]:
parameters = {'layers': [],
                   'activations': [['relu']],
                   'dropout': [[0.05], [0.15], [0.25]],
                   'attention': [128],
             'input_shape': [document_matrix.iloc[0:5].shape[1]], 
              'nb_classes': [labels.iloc[0:5].shape[1]]}
for j in [64, 128, 256, 512, 1024, 2048]:
    for i in range(3):
        parameters['layers'].append([j]*(i+1))

In [11]:
parameters

{'activations': [['relu']],
 'attention': [128],
 'dropout': [[0.05], [0.15], [0.25]],
 'input_shape': [68],
 'layers': [[64],
  [64, 64],
  [64, 64, 64],
  [128],
  [128, 128],
  [128, 128, 128],
  [256],
  [256, 256],
  [256, 256, 256],
  [512],
  [512, 512],
  [512, 512, 512],
  [1024],
  [1024, 1024],
  [1024, 1024, 1024],
  [2048],
  [2048, 2048],
  [2048, 2048, 2048]],
 'nb_classes': [11]}

In [12]:
ffn_grid = GridSearchCV(estimator=ffn, param_grid=parameters, n_jobs=3, verbose=1)

In [13]:
grid_result = ffn_grid.fit(document_matrix, labels, epochs=100)

Fitting 3 folds for each of 54 candidates, totalling 162 fits


/mnt/c/Users/Thembanip/Documents/py/sa-language-classifier/sa_lang_env/lib/python3.5/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  4.0min
[Parallel(n_jobs=3)]: Done 162 out of 162 | elapsed: 30.2min finished


Epoch 1/10
1000/1000 [==============================] - 4s 4ms/step - loss: 1.8008 - acc: 0.4510
Epoch 2/10
1000/1000 [==============================] - 3s 3ms/step - loss: 0.8498 - acc: 0.7260
Epoch 3/10
1000/1000 [==============================] - 2s 2ms/step - loss: 0.6512 - acc: 0.7890
Epoch 4/10
1000/1000 [==============================] - 2s 2ms/step - loss: 0.5713 - acc: 0.7990
Epoch 5/10
1000/1000 [==============================] - 2s 2ms/step - loss: 0.5035 - acc: 0.8260
Epoch 6/10
1000/1000 [==============================] - 3s 3ms/step - loss: 0.4512 - acc: 0.8490
Epoch 7/10
1000/1000 [==============================] - 4s 4ms/step - loss: 0.4198 - acc: 0.8550
Epoch 8/10
1000/1000 [==============================] - 3s 3ms/step - loss: 0.3894 - acc: 0.8650
Epoch 9/10
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3661 - acc: 0.8790
Epoch 10/10
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3539 - acc: 0.8760


In [14]:
pd.DataFrame(grid_result.cv_results_).sort_values('mean_test_score', ascending=False)

/mnt/c/Users/Thembanip/Documents/py/sa-language-classifier/sa_lang_env/lib/python3.5/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/mnt/c/Users/Thembanip/Documents/py/sa-language-classifier/sa_lang_env/lib/python3.5/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/mnt/c/Users/Thembanip/Documents/py/sa-language-classifier/sa_lang_env/lib/python3.5/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_activations,param_attention,param_dropout,param_input_shape,param_layers,param_nb_classes,...,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
13,42.768974,1.487677,0.797,0.946000,[relu],128,[0.05],68,"[1024, 1024]",11,...,0.814371,0.945946,0.783784,0.938531,0.792793,0.953523,0.059467,0.085348,0.012839,0.006121
52,100.736751,4.622157,0.795,0.926000,[relu],128,[0.25],68,"[2048, 2048]",11,...,0.820359,0.926426,0.795796,0.917541,0.768769,0.934033,0.294189,0.500056,0.021074,0.006739
17,223.831091,2.026304,0.792,0.966505,[relu],128,[0.05],68,"[2048, 2048, 2048]",11,...,0.808383,0.977477,0.789790,0.961019,0.777778,0.961019,0.348028,0.060322,0.012595,0.007758
16,121.860950,1.745150,0.792,0.963001,[relu],128,[0.05],68,"[2048, 2048]",11,...,0.811377,0.965465,0.795796,0.967016,0.768769,0.956522,0.333983,0.029222,0.017605,0.004625
34,121.558473,3.723957,0.791,0.948996,[relu],128,[0.15],68,"[2048, 2048]",11,...,0.799401,0.941441,0.792793,0.946027,0.780781,0.959520,6.145709,0.740501,0.007708,0.007673
35,214.099664,4.318652,0.789,0.951494,[relu],128,[0.15],68,"[2048, 2048, 2048]",11,...,0.787425,0.939940,0.810811,0.959520,0.768769,0.955022,2.167750,0.806629,0.017191,0.008374
49,24.862370,3.771168,0.788,0.876497,[relu],128,[0.25],68,"[1024, 1024]",11,...,0.802395,0.870871,0.780781,0.880060,0.780781,0.878561,0.542790,0.303048,0.010194,0.004025
53,174.396906,5.391669,0.784,0.909499,[relu],128,[0.25],68,"[2048, 2048, 2048]",11,...,0.790419,0.906907,0.774775,0.911544,0.786787,0.910045,0.195293,0.837150,0.006685,0.001932
32,48.326230,2.711005,0.784,0.907499,[relu],128,[0.15],68,"[1024, 1024, 1024]",11,...,0.793413,0.905405,0.795796,0.904048,0.762763,0.913043,0.101066,0.192890,0.015037,0.003960
14,68.088104,1.612082,0.781,0.958498,[relu],128,[0.05],68,"[1024, 1024, 1024]",11,...,0.778443,0.954955,0.801802,0.953523,0.762763,0.967016,0.264536,0.111811,0.016032,0.006052


In [15]:
model = parameter_ffn(document_matrix, labels, **{'layers': [128, 128, 128],
                                                  'activations': ['relu'],
                                                  'dropout': [0.15],
                                                  'attention': 128})
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 68)           0                                            
__________________________________________________________________________________________________
dense_17 (Dense)                (None, 128)          8832        input_6[0][0]                    
__________________________________________________________________________________________________
activation_17 (Activation)      (None, 128)          0           dense_17[0][0]                   
__________________________________________________________________________________________________
dropout_12 (Dropout)            (None, 128)          0           activation_17[0][0]              
__________________________________________________________________________________________________
batch_norm

In [35]:
models = []
for i in training_data:
    print(i['document_matrix'].shape)
    model = simple_ffn(i['document_matrix'], i['labels'])
    model.fit(i['document_matrix'], i['labels'], epochs=100, validation_split=0.1)
    models.append(model)

(10139, 68)
Train on 9125 samples, validate on 1014 samples
Epoch 1/100
9125/9125 [==============================] - 1s 131us/step - loss: 2.3898 - acc: 0.2655 - val_loss: 2.3824 - val_acc: 0.3402
Epoch 2/100
9125/9125 [==============================] - 1s 89us/step - loss: 2.3738 - acc: 0.3468 - val_loss: 2.3614 - val_acc: 0.3205
Epoch 3/100
9125/9125 [==============================] - 1s 89us/step - loss: 2.3360 - acc: 0.3395 - val_loss: 2.3005 - val_acc: 0.3323
Epoch 4/100
9125/9125 [==============================] - 1s 68us/step - loss: 2.2720 - acc: 0.3530 - val_loss: 2.2307 - val_acc: 0.3728
Epoch 5/100
9125/9125 [==============================] - 1s 69us/step - loss: 2.2051 - acc: 0.3784 - val_loss: 2.1467 - val_acc: 0.3895
Epoch 6/100
9125/9125 [==============================] - 1s 69us/step - loss: 2.1287 - acc: 0.4169 - val_loss: 2.0746 - val_acc: 0.4783
Epoch 7/100
9125/9125 [==============================] - 1s 134us/step - loss: 2.0666 - acc: 0.4671 - val_loss: 2.0086 - va

9125/9125 [==============================] - 4s 432us/step - loss: 1.2055 - acc: 0.6450 - val_loss: 1.0463 - val_acc: 0.7426
Epoch 20/100
9125/9125 [==============================] - 4s 420us/step - loss: 1.1855 - acc: 0.6412 - val_loss: 1.0136 - val_acc: 0.7406
Epoch 21/100
9125/9125 [==============================] - 4s 449us/step - loss: 1.1567 - acc: 0.6401 - val_loss: 0.9818 - val_acc: 0.7396
Epoch 22/100
9125/9125 [==============================] - 4s 436us/step - loss: 1.1214 - acc: 0.6462 - val_loss: 0.9541 - val_acc: 0.7396
Epoch 23/100
9125/9125 [==============================] - 4s 483us/step - loss: 1.0945 - acc: 0.6503 - val_loss: 0.9288 - val_acc: 0.7377
Epoch 24/100
9125/9125 [==============================] - 4s 399us/step - loss: 1.0878 - acc: 0.6376 - val_loss: 0.9026 - val_acc: 0.7367
Epoch 25/100
9125/9125 [==============================] - 4s 432us/step - loss: 1.0638 - acc: 0.6412 - val_loss: 0.8742 - val_acc: 0.7377
Epoch 26/100
9125/9125 [=======================

9125/9125 [==============================] - 38s 4ms/step - loss: 0.9890 - acc: 0.6283 - val_loss: 0.8459 - val_acc: 0.6874
Epoch 38/100
9125/9125 [==============================] - 36s 4ms/step - loss: 0.9684 - acc: 0.6302 - val_loss: 0.8334 - val_acc: 0.6874
Epoch 39/100
9125/9125 [==============================] - 36s 4ms/step - loss: 0.9759 - acc: 0.6238 - val_loss: 0.8238 - val_acc: 0.6874
Epoch 40/100
9125/9125 [==============================] - 36s 4ms/step - loss: 0.9650 - acc: 0.6307 - val_loss: 0.8167 - val_acc: 0.6874
Epoch 41/100
9125/9125 [==============================] - 38s 4ms/step - loss: 0.9600 - acc: 0.6342 - val_loss: 0.8066 - val_acc: 0.6864
Epoch 42/100
9125/9125 [==============================] - 37s 4ms/step - loss: 0.9463 - acc: 0.6327 - val_loss: 0.8013 - val_acc: 0.6874
Epoch 43/100
9125/9125 [==============================] - 38s 4ms/step - loss: 0.9326 - acc: 0.6384 - val_loss: 0.7920 - val_acc: 0.6874
Epoch 44/100
9125/9125 [==============================

Epoch 97/100
9125/9125 [==============================] - 48s 5ms/step - loss: 0.6554 - acc: 0.7082 - val_loss: 0.6791 - val_acc: 0.7347
Epoch 98/100
9125/9125 [==============================] - 38s 4ms/step - loss: 0.6547 - acc: 0.7054 - val_loss: 0.6738 - val_acc: 0.7357
Epoch 99/100
9125/9125 [==============================] - 38s 4ms/step - loss: 0.6434 - acc: 0.7110 - val_loss: 0.6735 - val_acc: 0.7387
Epoch 100/100
9125/9125 [==============================] - 40s 4ms/step - loss: 0.6421 - acc: 0.7144 - val_loss: 0.6731 - val_acc: 0.7367


In [23]:
model.save(os.path.join(MODELS_PATH,'ffn_sample_model_sentences_bi.h5'))

## Test on unseen data 

In [36]:
test_data = []
for i in training_data:
    X_test, y_test = prepare_test_data(corpora_test, labels_test, i['pipeline_instance'])
    test_data.append({'X_test': X_test, 'y_test': y_test})

In [38]:
test_data[0]['X_test'].head()

,,!,"""",#,$,%,&,',(,),...,u,v,w,x,y,z,{,|,},~
0,0.351427,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.080407,0.080444,...,0.198643,0.000000,0.168340,0.118161,0.178917,0.000000,0.0,0.0,0.0,0.0
1,0.531607,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.037561,0.000000,0.038198,0.268115,0.121792,0.000000,0.0,0.0,0.0,0.0
2,0.470930,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.038955,0.054692,0.066025,0.000000,0.014035,0.000000,0.0,0.0,0.0,0.0
3,0.231594,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.087272,0.000000,0.266251,0.000000,0.188653,0.151879,0.0,0.0,0.0,0.0
4,0.380765,0.0,0.0,0.0,0.0,0.0,0.0,0.075985,0.065340,0.065370,...,0.336291,0.018886,0.054718,0.000000,0.087234,0.070230,0.0,0.0,0.0,0.0


In [39]:
test_data[0]['y_test'].head()

,af,en,nr,nso,ss,st,tn,ts,ve,xh,zu
34750,0,0,0,0,0,0,0,0,0,1,0
18986,0,0,0,1,0,0,0,0,0,0,0
13655,0,1,0,0,0,0,0,0,0,0,0
15126,0,0,1,0,0,0,0,0,0,0,0
21978,0,0,0,0,1,0,0,0,0,0,0


In [42]:
for idx, i in enumerate(test_data):
    score, accuracy = models[idx].evaluate(i['X_test'], i['y_test'])
    print('Model test accuracy', accuracy.round(4)*100)

30417/30417 [==============================] - 1s 28us/step
Model test accuracy 77.2
30417/30417 [==============================] - 2s 51us/step
Model test accuracy 75.77000000000001
30417/30417 [==============================] - 15s 504us/step
Model test accuracy 71.95
